# Kaggle Agents - Real Competition (Colab)

Este notebook executa o `kaggle-agents` em **competicoes reais do Kaggle**, baixando dados via API, gerando submissions automaticamente, e submetendo ao leaderboard.

Workflow:
1. Download dos dados via Kaggle API
2. Execucao autonoma do pipeline (search, plan, develop, ensemble, submit)
3. Auto-submit da `submission.csv` ao Kaggle
4. Visualizacao do score no leaderboard

Pre-requisitos (Colab Secrets):
- `OPENAI_API_KEY` ou `GEMINI_API_KEY` (dependendo do provider)
- `KAGGLE_USERNAME`
- `KAGGLE_KEY`

## 1) Setup (clone + install)

Clona o repositorio e instala as dependencias. Nao precisa do `mle-bench`.

In [ ]:
# GPU (opcional)
!nvidia-smi || echo "No GPU available (CPU mode)"

# Clone repo
!test -d /content/kaggle-agents || git clone https://github.com/gustavogomespl/kaggle-agents.git /content/kaggle-agents

%cd /content/kaggle-agents
!ls -la | head

In [ ]:
# Instalar deps (sem mle-bench)
!pip -q install uv
!uv pip install --system -e /content/kaggle-agents

import sys
print('python:', sys.version)

## 2) Configuracao (.env + Kaggle credentials)

Configure no Colab -> **Secrets**:
- `OPENAI_API_KEY` (ou `GEMINI_API_KEY`)
- `KAGGLE_USERNAME`
- `KAGGLE_KEY`

Opcional (para controle de custo/qualidade):
- `LLM_MODEL`, `PLANNER_MODEL`, `DEVELOPER_MODEL`, `EVALUATOR_MODEL`

In [ ]:
import os, json
from pathlib import Path

from google.colab import userdata

# ===== LLM config (ajuste aqui) =====
os.environ['LLM_PROVIDER'] = os.environ.get('LLM_PROVIDER', 'gemini')
os.environ['LLM_MODEL'] = os.environ.get('LLM_MODEL', 'gemini-3-flash-preview')
os.environ['LLM_TEMPERATURE'] = os.environ.get('LLM_TEMPERATURE', '0')
os.environ['LLM_MAX_TOKENS'] = os.environ.get('LLM_MAX_TOKENS', '16000')

# Per-role overrides (opcional)
os.environ['PLANNER_MODEL'] = os.environ.get('PLANNER_MODEL', os.environ['LLM_MODEL'])
os.environ['DEVELOPER_MODEL'] = os.environ.get('DEVELOPER_MODEL', os.environ['LLM_MODEL'])
os.environ['EVALUATOR_MODEL'] = os.environ.get('EVALUATOR_MODEL', os.environ['LLM_MODEL'])

# ===== Secrets =====
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY') or ''
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY') or ''
kaggle_username = userdata.get('KAGGLE_USERNAME') or ''
kaggle_key = userdata.get('KAGGLE_KEY') or ''

if not kaggle_username or not kaggle_key:
    raise ValueError('Missing KAGGLE_USERNAME/KAGGLE_KEY (configure nos Colab Secrets).')

# Kaggle credentials
kaggle_path = Path.home() / '.kaggle' / 'kaggle.json'
kaggle_path.parent.mkdir(parents=True, exist_ok=True)
kaggle_path.write_text(json.dumps({'username': kaggle_username, 'key': kaggle_key}))
kaggle_path.chmod(0o600)

# .env
env_path = Path('/content/kaggle-agents/.env')
env_path.write_text(
    "\n".join([
        f"LLM_PROVIDER={os.environ['LLM_PROVIDER']}",
        f"LLM_MODEL={os.environ['LLM_MODEL']}",
        f"LLM_TEMPERATURE={os.environ['LLM_TEMPERATURE']}",
        f"LLM_MAX_TOKENS={os.environ['LLM_MAX_TOKENS']}",
        f"PLANNER_MODEL={os.environ['PLANNER_MODEL']}",
        f"DEVELOPER_MODEL={os.environ['DEVELOPER_MODEL']}",
        f"EVALUATOR_MODEL={os.environ['EVALUATOR_MODEL']}",
        f"OPENAI_API_KEY={os.environ['OPENAI_API_KEY']}",
        f"KAGGLE_USERNAME={kaggle_username}",
        f"KAGGLE_KEY={kaggle_key}",
        f"GOOGLE_API_KEY={os.environ['GOOGLE_API_KEY']}",
        "LOG_LEVEL=INFO",
        "LOG_DIR=./logs",
    ]) + "\n"
)

print('kaggle.json:', kaggle_path)
print('.env:', env_path)
print('LLM_MODEL:', os.environ['LLM_MODEL'])

In [ ]:
# LangSmith Tracing (opcional)
print("Configuring LangSmith Tracing...")

try:
    from google.colab import userdata
    langchain_api_key = userdata.get('LANGSMITH')
    langsmith_project = userdata.get('LANGSMITH_PROJECT')
    print("Loaded secrets from Colab")
except Exception:
    langchain_api_key = None
    langsmith_project = None

if langchain_api_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
    os.environ["LANGCHAIN_PROJECT"] = langsmith_project or "kaggle-agents"
    print(f"LangSmith Tracing Enabled (Project: {os.environ['LANGCHAIN_PROJECT']})")

    if os.path.exists(".env"):
        with open(".env", "a") as f:
            f.write(f"\nLANGCHAIN_TRACING_V2=true\n")
            f.write(f"LANGCHAIN_ENDPOINT=https://api.smith.langchain.com\n")
            f.write(f"LANGCHAIN_API_KEY={langchain_api_key}\n")
            f.write(f"LANGCHAIN_PROJECT={os.environ['LANGCHAIN_PROJECT']}\n")
else:
    print("LangSmith Tracing Disabled (no LANGSMITH secret found)")

## 3) Competicao

Defina o slug da competicao Kaggle. Exemplos:
- `titanic` (Getting Started, classificacao binaria)
- `house-prices-advanced-regression-techniques` (Getting Started, regressao)
- `digit-recognizer` (Getting Started, classificacao multi-classe)
- `spaceship-titanic` (Getting Started, classificacao binaria)

**Importante:** Aceite as regras da competicao no site do Kaggle antes de rodar.

In [ ]:
COMPETITION = "titanic"  # slug da competicao Kaggle
MAX_ITERATIONS = 3
TIMEOUT = 3000  # timeout por componente em segundos

OUT_DIR = f"/content/kaggle_results/{COMPETITION}"

print(f"Competition: {COMPETITION}")
print(f"Max iterations: {MAX_ITERATIONS}")
print(f"Timeout: {TIMEOUT}s")
print(f"Output: {OUT_DIR}")

In [ ]:
# Rodar workflow + auto-submit
!KAGGLE_AGENTS_TARGET_SCORE=0.99 \
KAGGLE_AGENTS_MAX_COMPONENT_RETRIES=3 \
KAGGLE_AGENTS_COMPONENT_TIMEOUT_S=$TIMEOUT \
TESTING_TIMEOUT=$TIMEOUT \
TIMEOUT_MODEL_HEAVY=$TIMEOUT \
KAGGLE_AGENTS_MAX_COMPONENTS=6 \
KAGGLE_AGENTS_ENABLE_LIMITS=false \
KAGGLE_AGENTS_CV_FOLDS=5 \
python3 notebooks/kaggle_eval.py \
  --competition $COMPETITION \
  --max-iterations $MAX_ITERATIONS \
  --timeout $TIMEOUT \
  --auto-submit \
  -o $OUT_DIR

In [ ]:
# Ver resultados
import json
from pathlib import Path

results_path = Path(OUT_DIR) / "results.json"
summary_path = Path(OUT_DIR) / "summary.json"

if results_path.exists():
    results = json.loads(results_path.read_text())
    print("=" * 60)
    print("RESULTS")
    print("=" * 60)
    for r in results:
        print(f"\nCompetition: {r.get('competition_id')}")
        print(f"  Success: {r.get('success')}")
        print(f"  Submitted: {r.get('submitted')}")
        print(f"  Public Score: {r.get('public_score')}")
        print(f"  Execution Time: {r.get('execution_time', 0):.1f}s")
        print(f"  Iterations: {r.get('iterations')}")
        print(f"  Components: {r.get('components_implemented')}")
        if r.get('error'):
            print(f"  Error: {r.get('error')[:200]}")
else:
    print(f"No results found at {results_path}")

if summary_path.exists():
    summary = json.loads(summary_path.read_text())
    print("\n" + "=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print(json.dumps(summary, indent=2))

In [ ]:
# Ver leaderboard e posicao
import os
os.environ['KAGGLE_USERNAME'] = os.environ.get('KAGGLE_USERNAME', '')
os.environ['KAGGLE_KEY'] = os.environ.get('KAGGLE_KEY', '')

try:
    from kaggle_agents.tools.kaggle_api import KaggleAPIClient
    client = KaggleAPIClient()

    # My submissions
    print("=" * 60)
    print(f"MY SUBMISSIONS - {COMPETITION}")
    print("=" * 60)
    submissions = client.get_my_submissions(COMPETITION)
    for i, sub in enumerate(submissions[:5]):
        print(f"  [{i+1}] Score: {sub.get('publicScore')} | "
              f"Status: {sub.get('status')} | "
              f"Date: {sub.get('date')} | "
              f"File: {sub.get('fileName')}")

    # Leaderboard top 10
    print(f"\n{'=' * 60}")
    print(f"LEADERBOARD TOP 10 - {COMPETITION}")
    print("=" * 60)
    leaderboard = client.get_leaderboard(COMPETITION, top_n=10)
    for entry in leaderboard:
        print(f"  #{entry['rank']:>4} | {entry['score']:>10} | {entry['teamName']}")

    # My best score vs leaderboard
    if submissions:
        best_score = max(
            (s.get('publicScore', 0) for s in submissions if s.get('status') == 'complete'),
            default=None,
        )
        if best_score is not None and leaderboard:
            total_teams = len(client.get_leaderboard(COMPETITION, top_n=10000))
            rank_estimate = sum(1 for e in leaderboard if e['score'] >= best_score)
            percentile = (1 - rank_estimate / max(total_teams, 1)) * 100
            print(f"\nMy best score: {best_score}")
            print(f"Estimated rank: ~{rank_estimate}/{total_teams}")
            print(f"Estimated percentile: ~{percentile:.1f}%")

except Exception as e:
    print(f"Could not fetch leaderboard: {e}")